In [134]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from past.builtins import xrange

In [2]:
df = pd.read_csv('PAYX_dataset_2019Fall.csv')

In [3]:
df.head()

,Client_Id,A1,A2,A3,A4,A5,D1,B1,D2,A6,...,B5,E9,E10,E11,E12,E13,B6,Rep.Level,Size,Industry
0,1,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,Level 4,Size 3,Industry 10
1,2,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,Level 5,Size 4,Industry 6
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Level 5,Size 4,Industry 10
3,4,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,Level 4,Size 2,Industry 12
4,5,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,Level 4,Size 3,Industry 10


In [6]:
df_high = df.copy()

df_high['A'] = df[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']].sum(axis=1)
df_high['B'] = df[['B1', 'B2', 'B3', 'B4', 'B5', 'B6']].sum(axis=1)
df_high['C'] = df[['C1', 'C2', 'C3', 'C5', 'C6', 'C7']].sum(axis=1)
df_high['D'] = df[['D1', 'D2', 'D3', 'D4']].sum(axis=1)
df_high['E'] = df[['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13']].sum(axis=1)
df_high['F'] = df[['F1', 'F2']].sum(axis=1)

df_high.drop(columns=['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9',
                      'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'C1', 'C2', 'C3',
                      'C5', 'C6', 'C7', 'D1', 'D2', 'D3', 'D4','E1', 'E2',
                      'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12',
                      'E13', 'F1', 'F2'], inplace=True)

In [7]:
df_high.head()

,Client_Id,Rep.Level,Size,Industry,A,B,C,D,E,F
0,1,Level 4,Size 3,Industry 10,3,0,0,0,0,0
1,2,Level 5,Size 4,Industry 6,0,0,0,1,2,0
2,3,Level 5,Size 4,Industry 10,0,0,0,0,0,0
3,4,Level 4,Size 2,Industry 12,3,0,0,0,0,0
4,5,Level 4,Size 3,Industry 10,2,0,0,0,0,0


In [45]:
items = []
IDs2 = []
level2 = []
size2 = []
industry2 = [] 
weights = []

In [46]:
IDs = list(df_high['Client_Id'])
level = list(df_high['Rep.Level'])
size = list(df_high['Size'])
industry = list(df_high['Industry'])
A = list(df_high['A'])
B = list(df_high['B'])
C = list(df_high['C'])
D = list(df_high['D'])
E = list(df_high['E'])
F = list(df_high['F'])

In [47]:
for i in range(len(IDs)):
    if A[i] != 0:
        items.append('A')
        weights.append(A[i])
        IDs2.append(IDs[i])
        level2.append(level[i])
        size2.append(size[i])
        industry2.append(industry[i])
    if B[i] != 0:
        items.append('B')
        weights.append(B[i])
        IDs2.append(IDs[i])
        level2.append(level[i])
        size2.append(size[i])
        industry2.append(industry[i])
    if C[i] != 0:
        items.append('C')
        weights.append(C[i])
        IDs2.append(IDs[i])
        level2.append(level[i])
        size2.append(size[i])
        industry2.append(industry[i])
    if D[i] != 0:
        items.append('D')
        weights.append(D[i])
        IDs2.append(IDs[i])
        level2.append(level[i])
        size2.append(size[i])
        industry2.append(industry[i])
    if E[i] != 0:
        items.append('E')
        weights.append(E[i])
        IDs2.append(IDs[i])
        level2.append(level[i])
        size2.append(size[i])
        industry2.append(industry[i])
    if F[i] != 0:
        items.append('F')
        weights.append(F[i])
        IDs2.append(IDs[i])
        level2.append(level[i])
        size2.append(size[i])
        industry2.append(industry[i])

In [48]:
df_items = pd.DataFrame()
df_items['Client_Id'] = IDs2
df_items['Rep.Level'] = level2
df_items['Size'] = size2
df_items['Industry'] = industry2
df_items['Item'] = items
df_items['Weight'] = weights
df_items.head()

,Client_Id,Rep.Level,Size,Industry,Item,Weight
0,1,Level 4,Size 3,Industry 10,A,3
1,2,Level 5,Size 4,Industry 6,D,1
2,2,Level 5,Size 4,Industry 6,E,2
3,4,Level 4,Size 2,Industry 12,A,3
4,5,Level 4,Size 3,Industry 10,A,2


In [49]:
weight_mean_items = pd.DataFrame(df_items.groupby('Item')['Weight'].mean()).reset_index()
weight_mean_items.head()

,Item,Weight
0,A,2.139035
1,B,1.737308
2,C,1.420515
3,D,1.846770
4,E,1.371822


In [50]:
weight_mean_clients = df_items.groupby(by="Client_Id",as_index=False)['Weight'].mean()
weight_mean_clients.head()

,Client_Id,Weight
0,1,3.00
1,2,1.50
2,4,3.00
3,5,2.00
4,6,1.25


In [51]:
weight['number_of_weights'] = list(df_items.groupby('Item')['Weight'].count())
weight.sort_values('number_of_weights', ascending=False)

,Item,Weight,number_of_weights
5,F,1.059313,149512
4,E,1.371822,108880
3,D,1.846770,87078
0,A,2.139035,79239
2,C,1.420515,58552
1,B,1.737308,49697


In [52]:
item_matrix = df_items.pivot_table(index='Client_Id', columns='Item', values='Weight')
item_matrix = item_matrix.fillna(0)
item_matrix.head()

Item,A,B,C,D,E,F
Client_Id,,,,,,
1,3.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,2.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0
5,2.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,1.0,0.0,2.0,1.0


### KNN Content Based Model

In [94]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)

In [95]:
item_matrix.shape

(6, 277062)

In [104]:
item_matrix = item_matrix.T

In [105]:
item_matrix.shape

(6, 277062)

In [106]:
model_knn.fit(item_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=6, p=2, radius=1.0)

In [107]:
items = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F'}

In [108]:
item_matrix

Client_Id,1,2,4,5,6,8,9,10,11,12,...,386086,386087,386088,386089,386090,386091,386092,386094,386095,386096
Item,,,,,,,,,,,,,,,,,,,,,
A,3.0,0.0,3.0,2.0,1.0,0.0,0.0,0.0,3.0,0.0,...,2.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,0.0
B,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
D,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
E,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,2.0,3.0,0.0,1.0,1.0,1.0,3.0,1.0
F,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0


In [111]:
for i in items:
    print('For Item ', items[i], ": ")
    distances, indices = model_knn.kneighbors(item_matrix.iloc[i, :].values.reshape(1, -1), n_neighbors=6)
    count = 0
    for j in indices[0]:
        print('Distance to ', items[j], ' is ', round(distances[0][count], 3))
        count += 1

For Item  A : 
Distance to  A  is  0.0
Distance to  B  is  0.657
Distance to  D  is  0.667
Distance to  F  is  0.712
Distance to  E  is  0.727
Distance to  C  is  0.816
For Item  B : 
Distance to  B  is  0.0
Distance to  A  is  0.657
Distance to  D  is  0.698
Distance to  C  is  0.701
Distance to  E  is  0.71
Distance to  F  is  0.76
For Item  C : 
Distance to  C  is  0.0
Distance to  E  is  0.537
Distance to  B  is  0.701
Distance to  D  is  0.716
Distance to  F  is  0.73
Distance to  A  is  0.816
For Item  D : 
Distance to  D  is  0.0
Distance to  E  is  0.587
Distance to  F  is  0.614
Distance to  A  is  0.667
Distance to  B  is  0.698
Distance to  C  is  0.716
For Item  E : 
Distance to  E  is  0.0
Distance to  C  is  0.537
Distance to  D  is  0.587
Distance to  F  is  0.638
Distance to  B  is  0.71
Distance to  A  is  0.727
For Item  F : 
Distance to  F  is  0.0
Distance to  D  is  0.614
Distance to  E  is  0.638
Distance to  A  is  0.712
Distance to  C  is  0.73
Distance to  B  i

### Collaberating Filtering

In [112]:
df_high.head()

,Client_Id,Rep.Level,Size,Industry,A,B,C,D,E,F
0,1,Level 4,Size 3,Industry 10,3,0,0,0,0,0
1,2,Level 5,Size 4,Industry 6,0,0,0,1,2,0
2,3,Level 5,Size 4,Industry 10,0,0,0,0,0,0
3,4,Level 4,Size 2,Industry 12,3,0,0,0,0,0
4,5,Level 4,Size 3,Industry 10,2,0,0,0,0,0


In [113]:
df_high = df_high[['A', 'B', 'C', 'D', 'E', 'F']]

In [114]:
df_high.head()

,A,B,C,D,E,F
0,3,0,0,0,0,0
1,0,0,0,1,2,0
2,0,0,0,0,0,0
3,3,0,0,0,0,0
4,2,0,0,0,0,0


In [115]:
magnitude = np.sqrt(np.square(df_high).sum(axis=1))

In [116]:
df_high = df_high.divide(magnitude, axis ='index')

In [117]:
df_high = df_high.fillna(0)

In [118]:
df_high.head()

,A,B,C,D,E,F
0,1.0,0.0,0.0,0.000000,0.000000,0.0
1,0.0,0.0,0.0,0.447214,0.894427,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.0
3,1.0,0.0,0.0,0.000000,0.000000,0.0
4,1.0,0.0,0.0,0.000000,0.000000,0.0


In [119]:
def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

In [124]:
data_matrix = calculate_similarity(df_high)
data_matrix

,A,B,C,D,E,F
A,1.000000,0.191322,0.088641,0.186369,0.137852,0.131934
B,0.191322,1.000000,0.152221,0.151718,0.147013,0.106498
C,0.088641,0.152221,1.000000,0.141929,0.266853,0.133939
D,0.186369,0.151718,0.141929,1.000000,0.226685,0.195759
E,0.137852,0.147013,0.266853,0.226685,1.000000,0.200872
F,0.131934,0.106498,0.133939,0.195759,0.200872,1.000000


In [125]:
print(data_matrix.loc['A'].nlargest(6))

A    1.000000
B    0.191322
D    0.186369
E    0.137852
F    0.131934
C    0.088641
Name: A, dtype: float64


In [126]:
print(data_matrix.loc['B'].nlargest(6))

B    1.000000
A    0.191322
C    0.152221
D    0.151718
E    0.147013
F    0.106498
Name: B, dtype: float64


In [127]:
print(data_matrix.loc['C'].nlargest(6))

C    1.000000
E    0.266853
B    0.152221
D    0.141929
F    0.133939
A    0.088641
Name: C, dtype: float64


In [128]:
print(data_matrix.loc['D'].nlargest(6))

D    1.000000
E    0.226685
F    0.195759
A    0.186369
B    0.151718
C    0.141929
Name: D, dtype: float64


In [129]:
print(data_matrix.loc['E'].nlargest(6))

E    1.000000
C    0.266853
D    0.226685
F    0.200872
B    0.147013
A    0.137852
Name: E, dtype: float64


In [130]:
print(data_matrix.loc['F'].nlargest(6))

F    1.000000
E    0.200872
D    0.195759
C    0.133939
A    0.131934
B    0.106498
Name: F, dtype: float64


In [131]:
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,6))

In [135]:
for i in xrange(0, len(data_matrix.columns)):
    data_neighbours.ix[i,:5] = data_matrix.ix[0:,i].sort_values(ascending=False)[:5].index

In [136]:
data_neighbours

,1,2,3,4,5
A,A,B,D,E,F
B,B,A,C,D,E
C,C,E,B,D,F
D,D,E,F,A,B
E,E,C,D,F,B
F,F,E,D,C,A


In [137]:
client = 273497
client_index = df[df['Client_Id']==client].index.tolist()[0]

In [139]:
known_user_likes = df_high.ix[client_index]
known_user_likes = known_user_likes[known_user_likes >0].index.values

known_user_likes

array(['D', 'E', 'F'], dtype=object)

In [140]:
# Construct the neighbourhood from the most similar items to the
# ones our user has already liked.
most_similar_to_likes = data_neighbours.ix[known_user_likes]
similar_list = most_similar_to_likes.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))
neighbourhood = data_matrix[similar_list].ix[similar_list]

In [142]:
# A user vector containing only the neighbourhood items and
# the known user likes.
user_vector = df_high.ix[client_index].ix[similar_list]

# Calculate the score.
score = neighbourhood.dot(user_vector).div(neighbourhood.sum(axis=1))

# Drop the known likes.
score = score.drop(known_user_likes)

print(known_user_likes)
print(score.nlargest(2))

['D' 'E' 'F']
C    0.177826
A    0.149832
dtype: float64
